In [1]:
# Code from spherical pendulum
# import libraries
import sympy as sym
import numpy as np

from IPython.display import display #for pretty printing

# system parameters
X0,Y0, Z0 = sym.symbols(['X0','Y0', 'Z0']) # fixed position of Arm
g = sym.symbols('g')
mP,mA = sym.symbols(['mP','mA']) # mass of links
La,Lp = sym.symbols(['La','Lp']) # length of links
InA,InP = sym.symbols(['InA','InP']) # moment of intertia of links

# generalized coordinates
th1,ph1,th2,ph2 = sym.symbols(['\\theta_{1}','\\phi_{1}','\\theta_{2}','\\phi_{2}']) #position
dth1,dph1, dth2, dph2 = sym.symbols(['\\dot{\\theta_{1}}','\\dot{\\phi_{1}}', '\\dot{\\theta_{2}}','\\dot{\\phi_{2}}']) #velocity
ddth1,ddph1, ddth2, ddph2 = sym.symbols(['\\ddot{\\theta_{1}}','\\ddot{\\phi_{1}}', '\\ddot{\\theta_{2}}','\\ddot{\\phi_{2}}']) #acceleration

# assuming not absolute angles
thr = th1+th2
phr = ph1+ph2

q = sym.Matrix([[th1],[ph1],[th2],[ph2]]) #group into matrices
dq = sym.Matrix([[dth1],[dph1],[dth2],[dph2]])
ddq = sym.Matrix([[ddth1],[ddph1],[ddth2],[ddph2]])

qt = ddq.T
display(qt) #display prints it as cool latex stuff

# positiions
## Link 1
x1 = La*sym.sin(th1)*sym.cos(ph1)/2
y1 = -La*sym.sin(th1)*sym.sin(ph1)/2
z1 = -La*sym.cos(th1)/2

x2 = x1*2 + La*sym.sin(thr)*sym.cos(phr)/2
y2 = y1*2 - La*sym.sin(thr)*sym.sin(phr)/2
z2 = z1*2 - Lp*sym.cos(thr)/2

rArm = sym.simplify(sym.Matrix([x1, y1, z1]))
rPen = sym.simplify(sym.Matrix([x2, y2, z2]))


# velocities
## linear velocities
drArm = rArm.jacobian(q)*dq
drPen = rPen.jacobian(q)*dq

#display(drArm, drPen)
## angular velocities

# energies
## kinetic
Tarm = 1/2 * mA * drArm.T*drArm
Tpen = 1/2 * mP * drPen.T*drPen
T = sym.Matrix([sym.simplify(Tarm+Tpen)])


## potential
Varm = mA*g*z1
Vpen = mP*g*z2
V = sym.Matrix([sym.simplify(Varm+Vpen)])

# Lagrangian
Lg1 = sym.zeros(1,len(q))
for i in range(len(q)):
    dT_ddq = sym.diff(T,dq[i]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = T.jacobian(q) # partial of T in q

# term 4
Lg4 = V.jacobian(q) # partial of U in q

Matrix([[\ddot{\theta_{1}}, \ddot{\phi_{1}}, \ddot{\theta_{2}}, \ddot{\phi_{2}}]])

In [9]:
# Generalised forces
# STEP 5: calculate generalized forces
# arbitrary force
Fx, Fy, Fz = sym.symbols(['Fx','Fy', 'Fz'])

FF = sym.Matrix([[Fx],[Fy],[Fz]])

# end position in force coordinates wrt generalized coordinates:
x = sym.Matrix([[X0 + La*sym.sin(th1) + Lp*sym.sin(th1 + th2)],
                [Y0 - La*sym.cos(th1) - Lp*sym.cos(th1 + th2)],
                [Z0]])

x = sym.Matrix([[X0 + La*sym.sin(th1)*sym.cos(ph1)*0.001], 
                [Y0 - La*sym.sin(th1)*sym.sin(ph1)*0.001], 
                [Z0 - La*sym.cos(th1)*0.001]]) # force position is at the origin
JF = x.jacobian(q) # the jacobian

QF = JF.transpose()*FF
display(JF,QF)
# control torque
tau = sym.symbols('\\tau')

Qtau = sym.Matrix([[0],[tau]])
display(Qtau)
# Qall = QF + Qtau

# # put it all together
# EOM = Lg1 - Lg3 + Lg4 - Qall.transpose()

Matrix([
[ 0.001*La*cos(\phi_{1})*cos(\theta_{1}), -0.001*La*sin(\phi_{1})*sin(\theta_{1}), 0, 0],
[-0.001*La*sin(\phi_{1})*cos(\theta_{1}), -0.001*La*sin(\theta_{1})*cos(\phi_{1}), 0, 0],
[               0.001*La*sin(\theta_{1}),                                       0, 0, 0]])

Matrix([
[0.001*Fx*La*cos(\phi_{1})*cos(\theta_{1}) - 0.001*Fy*La*sin(\phi_{1})*cos(\theta_{1}) + 0.001*Fz*La*sin(\theta_{1})],
[                             -0.001*Fx*La*sin(\phi_{1})*sin(\theta_{1}) - 0.001*Fy*La*sin(\theta_{1})*cos(\phi_{1})],
[                                                                                                                  0],
[                                                                                                                  0]])

Matrix([
[   0],
[\tau]])